<img src="http://www.espol.edu.ec/sites/default/files/nuevaespol/logo_negro.png" width="301" height="58" />

#     FACULTAD DE INGENIER&Iacute;A EN ELECTRICIDAD Y COMPUTACI&Oacute;N
##   DIPLOMADO EN CIENCIA DE DATOS PARA LOS NEGOCIOS
###  Laboratorio: ElasticSearch

<h3><strong>Introducci&oacute;n</strong></h3>
<p><strong>Elasticsearch</strong></p>
<p>Elasticsearch es un producto que permite indexar y analizar en tiempo real grandes cantidades de datos de manera distribuida. Va un paso m&aacute;s all&aacute; de la b&uacute;squeda por texto gracias a un DSL y un Api para b&uacute;squedas m&aacute;s complicadas.</p>
<p>Elasticsearch permite almacenar documentos ( estructurados o no ) e indexar todos los campos de estos documentos en casi tiempo real.</p>
<p>Elasticsearch a diferencia de otros sistemas parecidos, no necesita declarar un esquema de la informaci&oacute;n que a&ntilde;adimos, pero para sacar mayor partido a la informaci&oacute;n tendremos que a&ntilde;adir los llamados <code>mappings</code>que funcionan m&aacute;s o menos como un <code>schema</code></p>

<p><strong>Conceptos Básicos</strong></p>

<h4 id="0021">Cluster</h4>
<p>Un cluster es un conjunto de uno o m&aacute;s nodos que mantienen toda la informaci&oacute;n de manera distribuida e indexada. Cada cluster est&aacute; identificado por un nombre, por defecto se llaman &laquo;elasticsearch&raquo;.</p>
<h4 id="0022">Node</h4>
<p>Un nodo es un server que forma parte de un cluster, almacena tu informaci&oacute;n y ayuda con las tareas de indexaci&oacute;n y b&uacute;squeda del cluster. Los nodos est&aacute;n identificados por un nombre tambi&eacute;n, pero en este caso cada nodo est&aacute; nombrado tras un personaje de Marvel.</p>
<p>Por defecto est&aacute;n configurados para ser parte de un cluster con el nombre de &laquo;elasticsearch&raquo;.</p>
<p>Puede haber tantos nodos como quieras por cada Cluster, en caso de que no haya ning&uacute;n Cluster configurado en el momento de creaci&oacute;n este lo crear&aacute; y se unir&aacute; a &eacute;l.</p>
<h4 id="0023">Index</h4>
<p>Un Index es una colecci&oacute;n de documentos que tienen caracter&iacute;sticas similares. Los &iacute;ndices est&aacute;n identificados por un nombre, el cual usaremos a la hora de indexar, buscar, actualizar y borrar.</p>
<h4 id="0024">Sharding y Replicas</h4>
<p>Cuando la informaci&oacute;n que estamos indexando sobrepasa el l&iacute;mite de una sola m&aacute;quina, <strong>elasticsearch</strong> nos ofrece distintas maneras de saltarnos esa limitaci&oacute;n.</p>
<p>El <strong>sharding</strong> nos permite dividir estos &iacute;ndices en distintas &laquo;piezas&raquo; ofreci&eacute;ndonos la posibilidad de <strong>escalar horizontalmente</strong> (a&ntilde;adiendo m&aacute;s m&aacute;quinas), adem&aacute;s de paralelizar y distribuir las distintas operaciones que hagamos sobre esos &iacute;ndices.</p>
<p>La <strong>replicaci&oacute;n</strong> nos ofrece un mecanismo para que en caso de fallo el usuario no se vea afectado.</p>

<p><strong>Descarga e Instalación</strong></p>

<p>Para instalar necesitamos tener como m&iacute;nimo la JDK 7 instalada en nuestro sistema.</p>
<p><a href="http://www.elasticsearch.org/overview/elkdownloads/">Link de descarga</a></p>
<p>Esperamos a que termine de iniciarse y accedemos a la URL ( http://localhost:9200/ ) para ver una respuesta en formato JSON.</p>

<h3>Procedimiento</h3>

<h3>Paso 1: Comprobación de Elasticsearch</h3>

In [1]:
# make sure ES is up and running
import requests
res = requests.get('http://localhost:9200')
print(res.content)

b'{\n  "name" : "783f82c48c70",\n  "cluster_name" : "docker-cluster",\n  "cluster_uuid" : "ubCYmKL-TBW79LbzQf1XKA",\n  "version" : {\n    "number" : "7.8.0",\n    "build_flavor" : "default",\n    "build_type" : "docker",\n    "build_hash" : "757314695644ea9a1dc2fecd26d1a43856725e65",\n    "build_date" : "2020-06-14T19:35:50.234439Z",\n    "build_snapshot" : false,\n    "lucene_version" : "8.5.1",\n    "minimum_wire_compatibility_version" : "6.8.0",\n    "minimum_index_compatibility_version" : "6.0.0-beta1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [3]:
#connect to our cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [19]:
#let's iterate over swapi people documents and index them
import json

r = requests.get('http://localhost:9200') 
i = 19
while r.status_code == 200:
    r = requests.get('https://swapi.dev/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1

print(i)

/Users/washington/opt/anaconda3/lib/python3.7/site-packages/elasticsearch/connection/base.py:177: ElasticsearchDeprecationWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


85


<p>Tenga en cuenta que automáticamente creamos un índice "sw" y un "doc_type" con el comando de indexación. Recibimos 18 respuestas de swapi y las indexamos con ES.</p>

<h3>Paso 2: Obtener un dato</h3>
<p>para ver si todo funcionó con estos pocos resultados, intentamos obtener el documento con id = 5.</p>

In [5]:
es.get(index='sw', doc_type='people', id=5)

{'_index': 'sw',
 '_type': 'people',
 '_id': '5',
 '_version': 1,
 '_seq_no': 4,
 '_primary_term': 1,
 'found': True,
 '_source': {'name': 'Leia Organa',
  'height': '150',
  'mass': '49',
  'hair_color': 'brown',
  'skin_color': 'light',
  'eye_color': 'brown',
  'birth_year': '19BBY',
  'gender': 'female',
  'homeworld': 'http://swapi.dev/api/planets/2/',
  'films': ['http://swapi.dev/api/films/1/',
   'http://swapi.dev/api/films/2/',
   'http://swapi.dev/api/films/3/',
   'http://swapi.dev/api/films/6/'],
  'species': [],
  'vehicles': ['http://swapi.dev/api/vehicles/30/'],
  'starships': [],
  'created': '2014-12-10T15:20:09.791000Z',
  'edited': '2014-12-20T21:17:50.315000Z',
  'url': 'http://swapi.dev/api/people/5/'}}

<strong>¿Donde está Darth Vader?</strong>

In [49]:
es.search(index="sw", body={"query": {"match": {'name':'Darth Vader'}}})

{'took': 11,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 6.384812,
  'hits': [{'_index': 'sw',
    '_type': 'people',
    '_id': '4',
    '_score': 6.384812,
    '_source': {'films': ['http://swapi.dev/api/films/1/',
      'http://swapi.dev/api/films/2/',
      'http://swapi.dev/api/films/3/',
      'http://swapi.dev/api/films/6/'],
     'homeworld': 'http://swapi.dev/api/planets/1/',
     'gender': 'male',
     'skin_color': 'white',
     'edited': '2014-12-20T21:17:50.313000Z',
     'created': '2014-12-10T15:18:20.704000Z',
     'mass': '136',
     'vehicles': [],
     'url': 'http://swapi.dev/api/people/4/',
     'hair_color': 'none',
     'birth_year': '41.9BBY',
     'eye_color': 'yellow',
     'species': [],
     'starships': ['http://swapi.dev/api/starships/13/'],
     'name': 'Darth Vader',
     'height': 208}},
   {'_index': 'sw',
    '_type': 'people',
    '_id': '

In [53]:
es.search(index="sw", body={"query": {"multi_match": { "query":"Darth Vader 136", "fields": [ "name", "mass" ] 
    }}})

{'took': 23,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3, 'relation': 'eq'},
  'max_score': 6.384812,
  'hits': [{'_index': 'sw',
    '_type': 'people',
    '_id': '4',
    '_score': 6.384812,
    '_source': {'films': ['http://swapi.dev/api/films/1/',
      'http://swapi.dev/api/films/2/',
      'http://swapi.dev/api/films/3/',
      'http://swapi.dev/api/films/6/'],
     'homeworld': 'http://swapi.dev/api/planets/1/',
     'gender': 'male',
     'skin_color': 'white',
     'edited': '2014-12-20T21:17:50.313000Z',
     'created': '2014-12-10T15:18:20.704000Z',
     'mass': '136',
     'vehicles': [],
     'url': 'http://swapi.dev/api/people/4/',
     'hair_color': 'none',
     'birth_year': '41.9BBY',
     'eye_color': 'yellow',
     'species': [],
     'starships': ['http://swapi.dev/api/starships/13/'],
     'name': 'Darth Vader',
     'height': 208}},
   {'_index': 'sw',
    '_type': 'people',
    '_id': '

In [48]:
es.search(index="sw", body={"query": {"prefix" : { "name" : "lu" }}})

{'took': 17,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'sw',
    '_type': 'people',
    '_id': '1',
    '_score': 1.0,
    '_source': {'name': 'Luke Skywalker',
     'height': '172',
     'mass': '77',
     'hair_color': 'blond',
     'skin_color': 'fair',
     'eye_color': 'blue',
     'birth_year': '19BBY',
     'gender': 'male',
     'homeworld': 'http://swapi.dev/api/planets/1/',
     'films': ['http://swapi.dev/api/films/1/',
      'http://swapi.dev/api/films/2/',
      'http://swapi.dev/api/films/3/',
      'http://swapi.dev/api/films/6/'],
     'species': [],
     'vehicles': ['http://swapi.dev/api/vehicles/14/',
      'http://swapi.dev/api/vehicles/30/'],
     'starships': ['http://swapi.dev/api/starships/12/',
      'http://swapi.dev/api/starships/22/'],
     'created': '2014-12-09T13:50:51.644000Z',
     'edited': '2014-12-20T21:17:56.8

In [20]:
es.search(index="sw", body={"query": {"prefix" : { "name" : "mol" }}})

{'took': 1118,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

Hay muchas otras consultas interesantes que podemos hacer. Si, por ejemplo, queremos obtener todos los elementos similares de alguna manera, para una búsqueda relacionada o de corrección podemos usar algo como esto:

In [26]:
es.search(index="sw", body={"query": {"more_like_this": { "fields" : ["name", "hair_color"], "like" : "black or Offee", "min_term_freq" : 1, "max_query_terms" : 12}}})

{'took': 18,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 12, 'relation': 'eq'},
  'max_score': 1.9527993,
  'hits': [{'_index': 'sw',
    '_type': 'people',
    '_id': '9',
    '_score': 1.9527993,
    '_source': {'name': 'Biggs Darklighter',
     'height': '183',
     'mass': '84',
     'hair_color': 'black',
     'skin_color': 'light',
     'eye_color': 'brown',
     'birth_year': '24BBY',
     'gender': 'male',
     'homeworld': 'http://swapi.dev/api/planets/1/',
     'films': ['http://swapi.dev/api/films/1/'],
     'species': [],
     'vehicles': [],
     'starships': ['http://swapi.dev/api/starships/12/'],
     'created': '2014-12-10T15:59:50.509000Z',
     'edited': '2014-12-20T21:17:50.323000Z',
     'url': 'http://swapi.dev/api/people/9/'}},
   {'_index': 'sw',
    '_type': 'people',
    '_id': '22',
    '_score': 1.9527993,
    '_source': {'name': 'Boba Fett',
     'height': '183',
     'mass': '78.2',

<h3>Paso 3: Actualizar Data</h3>

In [44]:
# update by index
def update_data_by_index(_index, _doc_type, _id, update_data):
    res = es.update(
        index=_index,
        doc_type=_doc_type,
        id=_id,
        body=update_data
    )
    return res


# update by query
def update_by_query(_index, query, field, update_data):
    _inline = "ctx._source.{field}={update_data}".format(field=field, update_data=update_data)
    _query = {
        "script": {
           "inline": _inline,
        },
        "query": {
           "match": query
        }
    }
    res = es.update_by_query(body=_query, index=_index)
    return res


# update by bulk api
def update_by_bulk(_index, _id, update_data, doc_type):
    action = [{
        "_id": _id,
        "_type": doc_type,
        "_index": _index,
        "_source": {"doc": update_data},
        "_op_type": 'update'
    }]
    res = helpers.bulk(es, action)
    return res


In [35]:
index = "sw"
_id = 5
_doc_type = "people"
update_data = {
    "doc": {'mass': '40'}
}
res = update_data_by_index(index, _doc_type, _id, update_data)
res

{'_index': 'sw',
 '_type': 'people',
 '_id': '5',
 '_version': 3,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 84,
 '_primary_term': 1}

In [36]:
es.get(index='sw', doc_type='people', id=5)

{'_index': 'sw',
 '_type': 'people',
 '_id': '5',
 '_version': 3,
 '_seq_no': 84,
 '_primary_term': 1,
 'found': True,
 '_source': {'name': 'Leia Organa',
  'height': '150',
  'mass': '40',
  'hair_color': 'brown',
  'skin_color': 'light',
  'eye_color': 'brown',
  'birth_year': '19BBY',
  'gender': 'female',
  'homeworld': 'http://swapi.dev/api/planets/2/',
  'films': ['http://swapi.dev/api/films/1/',
   'http://swapi.dev/api/films/2/',
   'http://swapi.dev/api/films/3/',
   'http://swapi.dev/api/films/6/'],
  'species': [],
  'vehicles': ['http://swapi.dev/api/vehicles/30/'],
  'starships': [],
  'created': '2014-12-10T15:20:09.791000Z',
  'edited': '2014-12-20T21:17:50.315000Z',
  'url': 'http://swapi.dev/api/people/5/'}}

In [45]:
query = {"name": "Darth Vader"}
field = "height"
update_data = 208
res = update_by_query(index, query, field, update_data)
res

{'took': 106,
 'timed_out': False,
 'total': 2,
 'updated': 2,
 'deleted': 0,
 'batches': 1,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [47]:
es.search(index="sw", body={"query": {"match": {'name':"Darth Vader"}}})

{'took': 41,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 6.384812,
  'hits': [{'_index': 'sw',
    '_type': 'people',
    '_id': '4',
    '_score': 6.384812,
    '_source': {'films': ['http://swapi.dev/api/films/1/',
      'http://swapi.dev/api/films/2/',
      'http://swapi.dev/api/films/3/',
      'http://swapi.dev/api/films/6/'],
     'homeworld': 'http://swapi.dev/api/planets/1/',
     'gender': 'male',
     'skin_color': 'white',
     'edited': '2014-12-20T21:17:50.313000Z',
     'created': '2014-12-10T15:18:20.704000Z',
     'mass': '136',
     'vehicles': [],
     'url': 'http://swapi.dev/api/people/4/',
     'hair_color': 'none',
     'birth_year': '41.9BBY',
     'eye_color': 'yellow',
     'species': [],
     'starships': ['http://swapi.dev/api/starships/13/'],
     'name': 'Darth Vader',
     'height': 208}},
   {'_index': 'sw',
    '_type': 'people',
    '_id': '

In [55]:
from elasticsearch import helpers
update_data = {"age": 27}
res = update_by_bulk(index, _id, update_data, _doc_type)
res

/Users/washington/opt/anaconda3/lib/python3.7/site-packages/elasticsearch/connection/base.py:177: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


(1, [])

In [56]:
es.get(index='sw', doc_type='people', id=5)

{'_index': 'sw',
 '_type': 'people',
 '_id': '5',
 '_version': 4,
 '_seq_no': 89,
 '_primary_term': 1,
 'found': True,
 '_source': {'name': 'Leia Organa',
  'height': '150',
  'mass': '40',
  'hair_color': 'brown',
  'skin_color': 'light',
  'eye_color': 'brown',
  'birth_year': '19BBY',
  'gender': 'female',
  'homeworld': 'http://swapi.dev/api/planets/2/',
  'films': ['http://swapi.dev/api/films/1/',
   'http://swapi.dev/api/films/2/',
   'http://swapi.dev/api/films/3/',
   'http://swapi.dev/api/films/6/'],
  'species': [],
  'vehicles': ['http://swapi.dev/api/vehicles/30/'],
  'starships': [],
  'created': '2014-12-10T15:20:09.791000Z',
  'edited': '2014-12-20T21:17:50.315000Z',
  'url': 'http://swapi.dev/api/people/5/',
  'age': 27}}

<h3>Paso 4: Eliminar Data</h3>

In [57]:
def delete_data_by_bulk(_index, _doc_type, _id):
    action = [
        {
            '_op_type': 'delete',
            '_index': _index,
            '_type': _doc_type,
            '_id': _id,
        }
    ]
    res = helpers.bulk(es, action)
    return res
  
  
# clear all index items
def delete_index(_index):
    res = es.indices.delete(index=_index, ignore=[400, 404])
    return res


# delete content but empty item exist
def delete_by_query(_index, query):
    res = es.delete_by_query(
        index=_index,
        body={"query": {"match": query}},
        _source=True
    )
    return res


# delet all item
def delete_one(_index, _id):
    res = es.delete(index=_index, id=_id)
    return res

In [58]:
_index = "sw"
_id = "5"
res = delete_one(_index, _id)

In [59]:
es.get(index='sw', doc_type='people', id=5)

NotFoundError: NotFoundError(404, '{"_index":"sw","_type":"people","_id":"5","found":false}')

<p>Algunas búsquedas mas:</p>

In [61]:
# you need search in particular field and keyword must be word, not letter
def full_text_search(keyword, index, field):
    res = es.search(
        index=index,
        body={
            "query":{
                "match": {
                    field: keyword
                }
            }
        }
    )
    return res


def regex_search(pat, index, field):
    res = es.search(
        index=index,
        body={
            "query":{
                "regexp": {
                    field: pat
                }
            }
        }
    )
    return res


def prefix_search(prefix, index, field):
    res = es.search(
        index=index,
        body={
            "query":{
                "prefix": {
                    field: prefix
                }
            }
        }
    )
    return res


if __name__ == "__main__":
    index = "sw"
    keyword = "name"
    field = "name"
    regex_pat = ".*?e.+"
    prefix = "lu"
    res = full_text_search(keyword, index, field)
    res = regex_search(regex_pat, index, field)
    res = prefix_search(prefix, index, field)
    print(res)

{'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'sw', '_type': 'people', '_id': '1', '_score': 1.0, '_source': {'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'skin_color': 'fair', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'homeworld': 'http://swapi.dev/api/planets/1/', 'films': ['http://swapi.dev/api/films/1/', 'http://swapi.dev/api/films/2/', 'http://swapi.dev/api/films/3/', 'http://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['http://swapi.dev/api/vehicles/14/', 'http://swapi.dev/api/vehicles/30/'], 'starships': ['http://swapi.dev/api/starships/12/', 'http://swapi.dev/api/starships/22/'], 'created': '2014-12-09T13:50:51.644000Z', 'edited': '2014-12-20T21:17:56.891000Z', 'url': 'http://swapi.dev/api/people/1/'}}, {'_index': 'sw', '_type': 'people', '_id': '64', '_score': 1.0, '_source

<h2>Bibliografía</h2>
<ul>
    <li><a href="https://www.elastic.co">https://www.elastic.co</a></li>
    <li><a href="https://www.adictosaltrabajo.com/2014/09/12/primeros-pasos-elasticsearch/">https://www.adictosaltrabajo.com/2014/09/12/primeros-pasos-elasticsearch/</a></li>
    <li><a href="https://medium.com/analytics-vidhya/quick-start-elasticsearch-with-python-a2578cd87339">https://medium.com/analytics-vidhya/quick-start-elasticsearch-with-python-a2578cd87339</a></li>
</ul>